In [ ]:
import os
import googlemaps
import pandas as pd
import numpy as np
import time
import pickle
#pd.set_option('display.max_column', None)
#pd.set_option('display.max_row', None)
#pd.set_option('max_colwidth', None)

``` GEOCODE API```

In [ ]:
#api_key = os.getenv()
gm = googlemaps.Client('API_KEY')
gm_result = gm.geocode('pizza near ipoh') # it returns the coordinate of location, not places
gm_result

```MAIN API```

In [ ]:
geoloc_geocode_place_api = 'API_KEY'
gm_geoloc = googlemaps.Client(key=geoloc_geocode_place_api)

```----TEST START---------```

In [ ]:
test = "pizza in Batu Pahat"
test_geo_place = gm_geoloc.places(test, radius=100)


In [ ]:
print(test_geo_place['results'][0]['plus_code']['compound_code'].split(' '))

# test_geo_place['results'] -- type list
# test_geo_place['results'][0] -- type dict

In [ ]:
negeri_district = test_geo_place['results'][0]['plus_code']['compound_code'].split(' ')
negeri = negeri_district[-1]
district = ' '.join(negeri_district[1:-1]).replace(',', ' ')

print(negeri_district)
print("NEGERI:", negeri, "DISTRICT:", district)

```---------TEST ENDED-----```

```
------SCRAP START-------
```

In [ ]:
# input required(test)
food_keyword = ['restaurant', 'mcdonald', 'restoran', 'pizza', 'kfc', 'warung', 'food', 'eat', 
                'seafood', 'cuisine', 'fast food',
                'supermarket', 'pasaraya', 'bakery', 'cafe',

                'marketplace', 'market', 
                
                'electronic store', 'furniture store', 'pet store', 'clothing', 
                
                'dentist', 'doctor', 'pharmacy', 'health']

negeri_keywords = ['Perak', 'Selangor', 'Pahang', 'Negeri Sembilan', 'Terengganu', 'Kedah', 'Kuala Lumpur', 'Lembah Klang',
                    'Perlis', 'Kelantan', 'Johor', 'Melaka', 'Pulau Pinang', 'Putrajaya']

johor_district = ['Batu Pahat','Johor Bahru','Kluang','Kota Tinggi','Kulai','Mersing','Muar','Pontian','Segamat','Tangkak',]
kedah_district = ['Daerah Baling','Daerah Bandar Baharu','Kota Setar','Kuala Muda','Kubang Pasu','Kulim','Langkawi','Padang Terap','Pendang','Pokok Sena','Sik','Yan',]
kelantan_district = ['Bachok','Gua Musang','Jeli','Kota Bharu','Kuala Krai','Machang','Pasir Mas','Pasir Puteh','Tanah Merah','Tumpat','Lojing',]
melaka_district = ['Alor Gajah','Melaka Tengah','Jasin',]
neg9_district = ['Jelebu','Jempol','Kuala Pilah','Port Dickson','Rembau','Tampin',]
pahang_district = ['Bentong','Bera','Cameron Highland','Jerantut','Kuantan','Lipis','Maran','Pekan','Raub','Rompin','Temerloh',]
pinang_district = ['Timur Laut','Barat Daya','Seberang Perai Utara','Seberang Perai Tengah','Seberang Perai Selatan',]
perak_district = ['Batang Padang','Hilir Perak','Hulu Perak','Kampar','Kerian','Kinta''Kuala Kangsar','Larut, Matang dan Selama',
                    'Manjung','Muallim','Perak Tengah','Daerah Bagan Datuk',]
selangor_district = ['Gombak','Hulu Langat','Hulu Selangor','Klang','Kuala Langat','Kuala Selangor','Petaling','Sabak Bernam','Sepang']
terengganu_district = ['Besut','Dungun','Hulu Terengganu','Kemaman','Kuala Terengganu','Marang','Setiu','Kuala Nerus']
kl_district = ['Bukit Bintang','Titiwangsa' ,'Setiawangsa ','Wangsa Maju','Batu','Kepong','Segambut','Lembah Pantai' ,'Seputeh','Bandar Tun Razak','Cheras' ]
perlis = ['perlis',]

In [ ]:
# collect the data firstss
all_info = []
for keyword_1 in food_keyword:
    for keyword in kl_district:

        input_location = "{0} in {1}".format(keyword_1, keyword)
        rad = 500
        full_information = gm_geoloc.places(input_location, radius=rad)
        full_info = full_information['results'] 
        all_info.append(full_info)

        time.sleep(3)

In [ ]:
with open ("raw_info_negeri_KL.txt", 'wb') as keep_in:
    pickle.dump(all_info, keep_in)

In [ ]:
with open ("raw_info_negeri_KL.txt", 'rb') as keep_in:
    all_info = pickle.load(keep_in)

In [ ]:
print(len(all_info[0]))

In [ ]:
# extracting the sepcific information
dict_info = {"Name":[], "Latitude":[], "Longitude":[], "Address": [], 
            "Price_level":[], "Rating":[], "Types":[], "Daerah":[], "Negeri":[]}

for loc_list in all_info: # data is a dict
    for data in loc_list:
        # name: 'name'
        name = data['name']
        # adress: 'formatted_address'
        address = data['formatted_address']
        # coordinate: 'geometry' --> 'input_location
        latitude = data['geometry']['location']['lat']
        longitude = data['geometry']['location']['lng']

        # negeri & daerah
        if 'plus_code' in data:
            negeri_district = data['plus_code']['compound_code'].split(' ')
            negeri = negeri_district[-1]

            # if NO daerah
            if len(negeri_district) <= 2:
                dict_info['Negeri'].append(negeri)
                dict_info['Daerah'].append(np.NaN)
            # if GOT daerah
            elif len(negeri_district) > 2:
                daerah = ' '.join(negeri_district[1:-1]).replace(',', '')
                dict_info['Negeri'].append(negeri)
                dict_info['Daerah'].append(daerah)

        else:
            dict_info["Negeri"].append(np.NaN)
            dict_info["Daerah"].append(np.NaN)

        # Price Level
        if 'price_level' in data:
            dict_info["Price_level"].append(data['price_level'])
        else:
            dict_info["Price_level"].append(np.NaN)
            
        # Rating
        if 'rating' in data:
            dict_info["Rating"].append(data['rating'])
        else:
            dict_info["Rating"].append(np.NaN)

        # Types/label
        if 'types' in data:
            dict_info["Types"].append(data['types'])
        else:
            dict_info["Types"].append(np.NaN)

        #appending
        dict_info["Name"].append(name)
        dict_info["Latitude"].append(str(latitude))
        dict_info["Longitude"].append(str(longitude))
        dict_info["Address"].append(address)

In [ ]:
print(len(dict_info['Daerah']))


``` PICKLE ```

In [ ]:
with open ("google_maps_scrap_KL.txt", 'wb') as keep_in:
    pickle.dump(dict_info, keep_in)

In [ ]:
with open ("google_maps_scrap_KL.txt", 'rb') as keep_in:
    dict_info = pickle.load(keep_in)

#dict_info


In [ ]:
print(len(dict_info['Name']))

```PICKLE END```

```FINAL DATAFRAME```

In [ ]:
df_loc = pd.DataFrame(dict_info, columns=dict_info.keys())
df_copy = df_loc.copy()

#df_loc

```
Pythonic method of LABELING PROCESS and locate it in new column
```

In [ ]:
def tagged_it(x):
    if "food" in x or 'restaurant' in x or 'bakery' in x or 'cafe' in x:
        return "food"
    elif 'dentist' in x or 'doctor' in x or 'health' in x:
        return 'health'
    elif 'electronic_store' in x or 'pet_store' in x or 'furniture_store' in x or 'pet_store' in x or 'clothing_store' in x:
        return 'store'
    elif 'supermarket' in x or 'shopping_mall' in x:
        return 'market'
    else:
        return 'others'

In [ ]:
df_copy['Label'] = df_copy['Types'].apply(lambda x: tagged_it(x))

```
Pythonic method end
```


In [ ]:
df_copy.head()

In [ ]:
df_copy['Label'].value_counts()

In [ ]:
df_copy.to_excel('gmaps-data-KL.xlsx')